In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jun 18 05:56:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 11.4 MB/s eta 0:00:00
   ━

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["test"] = load_dataset("Raziullah/librispeech_small_asr_fine-tune", split="test", use_auth_token=True)

print(common_voice)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2554: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/5567 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2620 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 2620
    })
})


In [5]:
common_voice = common_voice.remove_columns(["file", "speaker_id", "chapter_id", "id"])

print(common_voice)

DatasetDict({
    test: Dataset({
        features: ['audio', 'text'],
        num_rows: 2620
    })
})


In [6]:
dataset = common_voice

In [7]:
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 8.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.4 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=b7a6332a641e2825afedaffd7ec535ac0c1e1689e2193dbac2ad887743b13541
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper


In [8]:
import whisper
from datasets import DatasetDict, load_dataset
from jiwer import cer
import numpy as np

# Initialize the Whisper model
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:07<00:00, 19.7MiB/s]


In [13]:
def transcribe_and_calculate_cer(model, dataset):
    transcriptions = []
    cers = []
    i = 1
    for example in dataset['test']:
        # Get the audio file and the ground truth transcription
        audio = example['audio']['array']
        ground_truth = example['text']

        # Ensure audio data is in float32 format
        audio = np.array(audio, dtype=np.float32)  # Convert to float32

        # Transcribe the audio file
        print(f"Transcribing audio... {i}")
        result = model.transcribe(audio)
        transcription = result["text"]

        # Calculate CER
        error_rate = cer(ground_truth.lower(), transcription.lower())

        # Append results
        transcriptions.append(transcription)
        cers.append(error_rate)
        i += 1
    return dataset, transcriptions, cers

In [14]:
result_dataset, transcriptions, cers = transcribe_and_calculate_cer(model, dataset)

Transcribing audio... 1
Transcribing audio... 2
Transcribing audio... 3
Transcribing audio... 4
Transcribing audio... 5
Transcribing audio... 6
Transcribing audio... 7
Transcribing audio... 8
Transcribing audio... 9
Transcribing audio... 10
Transcribing audio... 11
Transcribing audio... 12
Transcribing audio... 13
Transcribing audio... 14
Transcribing audio... 15
Transcribing audio... 16
Transcribing audio... 17
Transcribing audio... 18
Transcribing audio... 19
Transcribing audio... 20
Transcribing audio... 21
Transcribing audio... 22
Transcribing audio... 23
Transcribing audio... 24
Transcribing audio... 25
Transcribing audio... 26
Transcribing audio... 27
Transcribing audio... 28
Transcribing audio... 29
Transcribing audio... 30
Transcribing audio... 31
Transcribing audio... 32
Transcribing audio... 33
Transcribing audio... 34
Transcribing audio... 35
Transcribing audio... 36
Transcribing audio... 37
Transcribing audio... 38
Transcribing audio... 39
Transcribing audio... 40
Transcrib

In [15]:
result_dataset['test'][0]

{'audio': {'path': '6930-75918-0000.flac',
  'array': array([-6.10351562e-05,  9.15527344e-05,  1.06811523e-03, ...,
         -2.19726562e-03, -1.13830566e-02, -8.81958008e-03]),
  'sampling_rate': 16000},
 'text': 'CONCORD RETURNED TO ITS PLACE AMIDST THE TENTS'}

In [17]:
cer = 0
for i in range(2620):
    print(f"Audio {i+1}:")
    print(f"Ground Truth: {result_dataset['test'][i]['text']}")
    print(f"Transcription: {transcriptions[i]}")
    print(f"CER: {cers[i]}\n")
    cer += cers[i]
print(f"Average CER: {cer/2620}")

Streaming output truncated to the last 5000 lines.
Ground Truth: YOU MUST SEE LIEUTENANT I SHOULD THINK THAT WE ARE NOT SO NEAR THE COAST OF ALGERIA AS YOU IMAGINED
Transcription:  You must see, Lieutenant, I should think that we are not so near the coast of Algeria as you imagined.
CER: 0.030303030303030304

Audio 1622:
Ground Truth: AFTER PONDERING AWHILE HE SAID IF WE WERE FARTHER AWAY I SHOULD EXPECT TO FIND A DEPTH OF TWO OR THREE HUNDRED FATHOMS INSTEAD OF FIVE FATHOMS FIVE FATHOMS
Transcription:  After pondering a while, he said, if we were farther away, I should expect to find a depth of two or three hundred fathoms instead of five fathoms. Five fathoms!
CER: 0.03870967741935484

Audio 1623:
Ground Truth: ITS DEPTH REMAINED INVARIABLE STILL FOUR OR AT MOST FIVE FATHOMS AND ALTHOUGH ITS BOTTOM WAS ASSIDUOUSLY DREDGED IT WAS ONLY TO PROVE IT BARREN OF MARINE PRODUCTION OF ANY TYPE
Transcription:  Its depth remained invariable, still four or at most five fathoms. And although its 

In [19]:

min_cer = min(cers)
print(f"BEST CER: {min_cer}")
print(f"Average CER: {cer/2620}")


BEST CER: 0.0
Average CER: 0.05920889288531899
